<a href="https://colab.research.google.com/github/ChenKua/xir/blob/main/robust04_Processed_DS_Haystack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Dataset

Connect to google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

Pickle file locations in my drive
<br>/content/drive/MyDrive/robust04/docs.pkl      containing document
<br>/content/drive/MyDrive/robust04/queries.pkl   containing queries
<br>/content/drive/MyDrive/robust04/qrels.pkl     containing query relevence
<br> For more detail of the datasets, please refer to https://ir-datasets.com/trec-robust04.html

<br>Note the official websit only offers .tar file.

In [4]:
# queries
queries_df = pd.read_pickle("/content/drive/MyDrive/robust04/queries.pkl")

In [ ]:
# documents
docs_df = pd.read_pickle("/content/drive/MyDrive/robust04/docs.pkl")

In [5]:
# query relevence
qrels_df = pd.read_pickle("/content/drive/MyDrive/robust04/qrels.pkl")

In [1]:
# Example
queries_df.head(2)

NameError: ignored

In [ ]:
docs_df.head(1)

,doc_id,text,marked_up_doc
0,FBIS3-1,"\n\nPOLITICIANS, PARTY PREFERENCES \n\n Sum...","<TEXT>\nPOLITICIANS, PARTY PREFERENCES \n\n ..."


In [ ]:
docs_df.shape

(528155, 3)

# Haystack





## Import

In [6]:
!pip install beir
!pip install tensorflow-text
!pip install farm-haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 4.6 MB/s 
     |████████████████████████████████| 85 kB 4.2 MB/s 
     |████████████████████████████████| 8.6 MB 75.5 MB/s 
     |████████████████████████████████| 219 kB 72.4 MB/s 
     |████████████████████████████████| 365 kB 81.6 MB/s 
     |████████████████████████████████| 212 kB 86.2 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 141 kB 89.7 MB/s 
     |████████████████████████████████| 115 kB 88.3 MB/s 
     |████████████████████████████████| 596 kB 87.7 MB/s 
     |████████████████████████████████| 127 kB 77.8 MB/s 
     |████████████████████████████████| 4.7 MB 83.7 MB/s 
     |████████████████████████████████| 1.3 MB 62.4 MB/s 
     |████████████████████████████████| 6.6 MB 76.0 MB/s 
  Created wheel for beir: filename=beir-1.0.1-py3-none-any.whl size=62516 sha256=6917a45215d6

In [7]:
from typing import List
import requests

from haystack import Document
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever
from haystack.utils import fetch_archive_from_http, print_answers, print_documents

from haystack.document_stores import FAISSDocumentStore
from haystack import Document

from haystack.modeling.evaluation.squad import compute_f1 as calculate_f1_str
from haystack.modeling.evaluation.squad import compute_exact as calculate_em_str

In [8]:
from haystack.retriever.dense import EmbeddingRetriever
from haystack.pipelines import DocumentSearchPipeline

In [9]:
import numpy as np

In [10]:
import pandas as pd

## Load into Document Store

In [ ]:
# optional if we have already the document store 
# run this code if writing new document
documents: List[Document] = []
for index, row in docs_df.iterrows():
  # if index == 10:
  #   break
  documents.append(Document(content=row['text'], meta={"name": row['doc_id'] or ""}))


In [ ]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", similarity="cosine", return_embedding=True)

In [ ]:
document_store.delete_documents()

document_store.write_documents(documents)
document_store.get_document_count()

Writing Documents:   0%|          | 0/528155 [00:00<?, ?it/s]

522815

In [11]:
!cp -R /content/drive/MyDrive/robust04/robust04 /content/robust04 

In [12]:
!cp -R /content/drive/MyDrive/robust04/robust04.json /content/robust04.json 

In [13]:
!cp -R /content/drive/MyDrive/robust04/FAISS_DS/faiss_document_store.db /content/faiss_document_store.db

In [ ]:
!cp -R /content/faiss_document_store.db /content/drive/MyDrive/robust04/FAISS_DS/
!cp -R /content/robust04.json /content/drive/MyDrive/robust04/FAISS_DS/robust04.json
!cp -R /content/robust04 /content/drive/MyDrive/robust04/FAISS_DS/robust04

In [14]:
document_store = FAISSDocumentStore.load("robust04", "robust04.json")
#document_store = FAISSDocumentStore.load("robust04")

In [ ]:
document_store.query_by_embedding( query_emb = np.zeros(768))

## Retriever

In [ ]:
retriever = EmbeddingRetriever(
   document_store=document_store,
   scale_score = False,
   embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
   model_format="sentence_transformers",
   batch_size = 128
)

In [ ]:
document_store.update_embeddings(retriever, update_existing_embeddings=False)
#document_store = FAISSDocumentStore.load("robust04")
# pick one of the two lines

Updating Embedding:   0%|          | 0/522815 [00:00<?, ? docs/s]

In [ ]:
document_store.save(index_path = "robust04") # only save after embedding

In [16]:
pipeline = DocumentSearchPipeline(retriever = retriever)

## Example retrieval

In [ ]:
index = 0
query = queries_df['title'][index]

In [ ]:
print(query)

International Organized Crime


In [ ]:
result = pipeline.run(
             query=query, 
             params={"Retriever": {"top_k": 100}}
             )

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result

In [ ]:
print(result["documents"][0].meta)
print(result["documents"][1].meta)
print(result["documents"][2].meta)
print(result["documents"][3].meta)
print(result["documents"][99].meta, result["documents"][99].score)

{'name': 'FBIS3-41285', 'vector_id': '40656'}
{'name': 'FBIS3-42315', 'vector_id': '41644'}
{'name': 'FBIS4-57987', 'vector_id': '117598'}
{'name': 'FBIS4-43552', 'vector_id': '103436'}
{'name': 'FBIS4-63104', 'vector_id': '122625'} 0.49457774


In [ ]:
temp_dic = {}
for i in range(100):
  temp_dic[result["documents"][i].meta["name"]] = result["documents"][i].score

In [ ]:
result["documents"][99].meta["name"]
result["documents"][99].score

0.49457774

In [ ]:
type(result["documents"][i].score)

numpy.float32

## Evaluation

In [ ]:
# https://github.com/cvangysel/pytrec_eval

In [17]:
!pip install pytrec_eval 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import pytrec_eval
import json

qrel = {
    'q1': {
        'd1': 0,
        'd2': 1,
        'd3': 0,
    },
    'q2': {
        'd2': 1,
        'd3': 1,
    },
}

run = {
    'q1': {
        'd1': 1.0,
        'd2': 0.0,
        'd3': 1.5,
    },
    'q2': {
        'd1': 1.5,
        'd2': 0.2,
        'd3': 0.5,
    }
}

evaluator = pytrec_eval.RelevanceEvaluator(
    qrel, {'map', 'ndcg','recall_100', 'P_100'})
# matrics can be found:  https://github.com/usnistgov/trec_eval/blob/de6a29f8ba9312c73f978aa9739695aa8ebf48eb/measures.c

print(json.dumps(evaluator.evaluate(run), indent=1))

{
 "q1": {
  "map": 0.3333333333333333,
  "P_100": 0.01,
  "recall_100": 1.0,
  "ndcg": 0.5
 },
 "q2": {
  "map": 0.5833333333333333,
  "P_100": 0.02,
  "recall_100": 1.0,
  "ndcg": 0.6934264036172708
 }
}


In [ ]:
type(run["q1"]["d3"])

float

In [ ]:
qrels_df.head()

,query_id,doc_id,relevance,iteration
0,301,FBIS3-10082,1,0
1,301,FBIS3-10169,0,0
2,301,FBIS3-10243,1,0
3,301,FBIS3-10319,0,0
4,301,FBIS3-10397,1,0


In [18]:
query_ids = qrels_df['query_id'].unique()

In [ ]:
query_ids

array(['301', '302', '303', '304', '305', '306', '307', '308', '309',
       '310', '311', '312', '313', '314', '315', '316', '317', '318',
       '319', '320', '321', '322', '323', '324', '325', '326', '327',
       '328', '329', '330', '331', '332', '333', '334', '335', '336',
       '337', '338', '339', '340', '341', '342', '343', '344', '345',
       '346', '347', '348', '349', '350', '351', '352', '353', '354',
       '355', '356', '357', '358', '359', '360', '361', '362', '363',
       '364', '365', '366', '367', '368', '369', '370', '371', '372',
       '373', '374', '375', '376', '377', '378', '379', '380', '381',
       '382', '383', '384', '385', '386', '387', '388', '389', '390',
       '391', '392', '393', '394', '395', '396', '397', '398', '399',
       '400', '401', '402', '403', '404', '405', '406', '407', '408',
       '409', '410', '411', '412', '413', '414', '415', '416', '417',
       '418', '419', '420', '421', '422', '423', '424', '425', '426',
       '427', '428',

In [20]:
# make qrel into double layer dic
qrels_dic = {}

for id in query_ids:
  query_id_1 = qrels_df[qrels_df["query_id"]==id]
  dic = pd.Series(query_id_1.relevance.values,index=query_id_1.doc_id).to_dict()
  qrels_dic[id] = dic

In [28]:
queries_df.head(100)

,query_id,title,description,narrative
0,301,International Organized Crime,"Identify organizations that participate in international criminal\nactivity,...",A relevant document must as a minimum identify the organization and the\ntyp...
1,302,Poliomyelitis and Post-Polio,Is the disease of Poliomyelitis (polio) under control in the\nworld?,Relevant documents should contain data or outbreaks of the \npolio disease (...
2,303,Hubble Telescope Achievements,Identify positive accomplishments of the Hubble telescope since it\nwas laun...,Documents are relevant that show the Hubble telescope has produced \nnew dat...
3,304,Endangered Species (Mammals),"Compile a list of mammals that are considered to be endangered,\nidentify th...",Any document identifying a mammal as endangered is relevant. \nStatements o...
4,305,Most Dangerous Vehicles,"Which are the most crashworthy, and least crashworthy, \npassenger vehicles?",A relevant document will contain information on the \ncrashworthiness of a g...
...,...,...,...,...
95,396,sick building syndrome,Identify documents that discuss sick building syndrome or\nbuilding-related ...,A relevant document would contain any data that refers to \nthe sick buildin...
96,397,automobile recalls,Identify documents that discuss the reasons for \nautomobile recalls.,A relevant document will specify major or minor reasons \nfor automobile rec...
97,398,dismantling Europe's arsenal,Identify documents that discuss the \nEuropean Conventional Arms Cut as it\n...,Relevant documents may address any of the\nfollowing issues: reducing conve...
98,399,oceanographic vessels,Identify documents that discuss the \nactivities or equipment of oceanograph...,Relevant documents will contain information on\nthe activity of oceanographi...


In [21]:
query_dic = pd.Series(queries_df.title.values,index=queries_df.query_id).to_dict()

In [ ]:
query_dic[str(301)]

'International Organized Crime'

In [22]:
retrieval_dic={}
for id in query_ids:
  print(query_dic[id])
  print(id)
  print("----------------- retrieval for id above")

  query = query_dic[id]
  result = pipeline.run(
             query=query, 
             params={"Retriever": {"top_k": 100}}
  )

  temp_dic = {}
  for i in range(100):
    temp_dic[result["documents"][i].meta["name"]] = result["documents"][i].score.astype(float)

  retrieval_dic[id] = temp_dic
    # result["documents"][99].meta["name"]
    # result["documents"][99].score

  print("----------------- retrieval for id above done")

International Organized Crime
301
----------------- retrieval for id above


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

----------------- retrieval for id above done
Poliomyelitis and Post-Polio
302
----------------- retrieval for id above


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

----------------- retrieval for id above done
Hubble Telescope Achievements
303
----------------- retrieval for id above


...

----------------- retrieval for id above done
term limits
699
----------------- retrieval for id above


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

----------------- retrieval for id above done
gasoline tax U.S.
700
----------------- retrieval for id above


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

----------------- retrieval for id above done


In [26]:
evaluator = pytrec_eval.RelevanceEvaluator(
    qrels_dic, {'map', 'ndcg','recall_100', 'P_100'})

print(json.dumps(evaluator.evaluate(retrieval_dic), indent=1))

{
 "301": {
  "map": 0.06752920686697206,
  "P_100": 0.43,
  "recall_100": 0.09598214285714286,
  "ndcg": 0.16671747200898543
 },
 "302": {
  "map": 0.21850329157453147,
  "P_100": 0.22,
  "recall_100": 0.3384615384615385,
  "ndcg": 0.4322333786891718
 },
 "303": {
  "map": 0.028307318307318308,
  "P_100": 0.05,
  "recall_100": 0.5,
  "ndcg": 0.198017282913974
 },
 "304": {
  "map": 0.01869135895952345,
  "P_100": 0.18,
  "recall_100": 0.09183673469387756,
  "ndcg": 0.11568276565855475
 },
 "305": {
  "map": 0.06839938674525141,
  "P_100": 0.08,
  "recall_100": 0.22857142857142856,
  "ndcg": 0.25639186771798667
 },
 "306": {
  "map": 0.01966871706807106,
  "P_100": 0.24,
  "recall_100": 0.07228915662650602,
  "ndcg": 0.10580336031671979
 },
 "307": {
  "map": 0.06537650154044505,
  "P_100": 0.37,
  "recall_100": 0.1761904761904762,
  "ndcg": 0.21331562878875918
 },
 "308": {
  "map": 0.3240914786967418,
  "P_100": 0.04,
  "recall_100": 1.0,
  "ndcg": 0.6318785275949567
 },
 "309": {
  